In [ ]:
import socket
import sys

In [ ]:
IP = '10.0.0.5'
PORT = 10001

In [ ]:
# Create a TCP/IP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Connect the socket to the port where the server is listening
server_address = (IP, PORT)
print >>sys.stderr, 'connecting to %s port %s' % server_address
sock.connect(server_address)

In [ ]:
# motor states
class State(object):
    def __init__(self,_name,_offset=0):
        self.name = _name
        self.fav_fc = 0
        self.ufav_fc = 0
        self.dcycle = zero
        self.offset = _offset
    def growth(self):
        self.ufav_fc = 0
        self.fav_fc = self.fav_fc + 1
        self.dcycle = min(max(self.dcycle,base) + (alpha*self.fav_fc) + self.offset,infinity)
    def decay(self):
        self.fav_fc = 0
        self.ufav_fc = self.ufav_fc + 1
        self.dcycle = max(self.dcycle - (beta*self.ufav_fc) -self.offset,zero)
    def get_value(self):
        return self.dcycle
    

In [ ]:
# params
zero = 0
infinity = 95
base = 40
alpha = 5
beta = 2
# global frame count
frame_count = 0

In [ ]:
l = State("left")
r = State("right")



In [ ]:
l.growth()
l.growth()
print l.get_value()
l.decay()
print l.get_value()

In [ ]:
import cv2
cap = cv2.VideoCapture("http://10.0.0.7:8080/video?x.mjpeg")

In [ ]:
try:
    message='x'
    while True:                
        ret, frame = cap.read()
        resized = cv2.resize(frame,(480,270), interpolation = cv2.INTER_CUBIC)
        cv2.imshow('frame',resized)
        
        key_press = cv2.waitKey(1)
        
        if key_press != message and key_press != -1:
            if 'w' == chr(key_press & 255):
                l.growth(); r.growth()
            elif 's' == chr(key_press & 255):
                l.decay(); r.decay()
            elif 'a' == chr(key_press & 255):
                r.growth(); l.decay()
            elif 'd' == chr(key_press & 255):
                l.growth(); r.decay()
            else:
                message='q'
            
            # construct message of form (l,r)
            message="(%s,%s)"%(l.get_value(),r.get_value())
            print >>sys.stderr, 'sending "%s"' % message
            sock.sendall(message)
        

        # Look for the response
        #amount_received = 0
        #amount_expected = len(message)
        '''
        while amount_received < amount_expected:
            data = sock.recv(16)
            amount_received += len(data)
            print >>sys.stderr, 'received "%s"' % data
        '''
finally:
    print >>sys.stderr, 'closing socket'
    sock.close()